In [10]:
# import libraries
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid

from scipy.integrate import odeint, solve_ivp
from scipy.fft import fft


import shap as shap
try:
    import lime
    import lime.lime_tabular    
except ImportError:
    pass

# Machine Learning Libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.utils import Sequence
from tensorflow import keras

# for reproducibility of this notebook:
rng = np.random.RandomState(42)
tf.random.set_seed(42)
np.random.seed(42)


import os
import sys


#idx = int(os.environ["SLURM_ARRAY_TASK_ID"])

#feature_setting = str(os.environ["Setting"])

#model_setting = str(os.environ["Model"])
idx = 0
model_setting = "True"
feature_setting = "Base"

"""
Define Parameter Configuration to Model

    Parameters
    ----------
    alpha : float, linear stiffness
    beta  : float, non linearity in the restoring force
    gamma : float, amplitude of the periodic driving force
    delta : float, amount of damping
    omega : float, angular frequency of the periodic driving force
"""   

parameter_list = [{'alpha' : 1.0, 'beta' : 1.0, 'gamma' : 0.37, 'delta' : 0.3, 'omega' : 1.2}, 
                  {'alpha' : 1.0, 'beta' : -0.5, 'gamma' : 0.37, 'delta' : 0.3, 'omega' : 1.2},
                  {'alpha' : 1.0, 'beta' : -0.5, 'gamma' : 0.37, 'delta' : 1.0, 'omega' : 1.2}, 
                  {'alpha' : 1.0, 'beta' : -0.5, 'gamma' : 0.5, 'delta' : 0.3, 'omega' : 1.2},
                  {'alpha' : 1.0, 'beta' : -0.5, 'gamma' : 0.37, 'delta' : 0.0, 'omega' : 1.2},
                  {'alpha' : -1.0, 'beta' : 1.0, 'gamma' : 0.37, 'delta' : 0.3, 'omega' : 1.2},
                  {'alpha' : -1.0, 'beta' : 1.0, 'gamma' : 0.37, 'delta' : 1.0, 'omega' : 1.2}, 
                  {'alpha' : -1.0, 'beta' : 1.0, 'gamma' : 0.5, 'delta' : 0.3, 'omega' : 1.2},
                  {'alpha' : -1.0, 'beta' : 1.0, 'gamma' : 0.0, 'delta' : 0.3, 'omega' : 0.0},
                  {'alpha' : -1.0, 'beta' : -1.0, 'gamma' : 0.37, 'delta' : 0.3, 'omega' : 1.2},
                  {'alpha' : 0.0, 'beta' : 0.0, 'gamma' : 0.37, 'delta' : 0.3, 'omega' : 1.2}]


dict_param = parameter_list[idx]

from OtherFunctions import *

if feature_setting == "Base":
    num_samples_ml = 100
    from  BaseDuffing import Duffing
elif feature_setting == "Random":
    num_samples_ml = 100
    from  RandomDuffing import Duffing
elif feature_setting == "Energy":
    num_samples_ml = 100
    from  EnergyDuffing import Duffing
elif feature_setting == "Gamma":
    num_samples_ml = 10
    from  GammaDuffing import Duffing


    


if __name__ == '__main__':
    duffing = Duffing(parameters = dict_param)
    eom = duffing.eom
    suffix = feature_setting + "_" + model_setting + "_" + duffing.suffix

    end_time = 100
    duffing.generate(num_samples_ml, samples = 10, end_time = end_time)
    duffing.scale_features()
    X_train, X_test, y_train, y_test = train_test_split(duffing.X_df[duffing.features], 
                                                        duffing.X_df[duffing.labels], test_size=0.1, random_state=42)
    
    X = X_test
    y = y_test
    
    # Create a basic model instance
    if model_setting == "Complex":
        model = MLModel()
        model_ = MLModel()
    elif model_setting == "Simple":
        model = SimpleModel()
        model_ = SimpleModel()
    elif model_setting == "True":
        model = duffing
        model_ = duffing.predict
        
    
    if (model_setting == "Simple") or (model_setting == "Complex"):
        """
        Train Model
        """
        callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=250),
                     tf.keras.callbacks.EarlyStopping(monitor='loss', patience=150)]


        history=model.fit(X_train, y_train, steps_per_epoch=None, epochs=5, validation_split=0.2, 
                          batch_size=1024, shuffle=True, callbacks=callbacks, verbose=0)


        model.save('Models/Model'+suffix)
        with open('Models/TrainingHistory/'+suffix, 'wb') as file_pi:
            pickle.dump(history.history, file_pi)
    
    def lime_x(X):
        if model_setting == "True":
            model = duffing
        return model.predict(X)[:,0]
    def lime_v(X):
        if model_setting == "True":
            model = duffing
        return model.predict(X)[:,1]
    

    explainers = ["kernel",  "lime", "numeric"]#"sampling",
    lime_models = [lime_x, lime_v]

    background = shap.sample(X_test, 10)
    choice = X.iloc[np.sort(np.random.choice(X_test.shape[0], 3, replace =False))]


    big_df = pd.DataFrame()
    for explainer in explainers:
        if explainer == "kernel":
            temp_explainer = shap.KernelExplainer(model_, background)
            temp_vals = temp_explainer.shap_values(choice)
        elif explainer == "sampling":
            temp_explainer = shap.SamplingExplainer(model_, background)
            temp_vals = temp_explainer.shap_values(choice)
        elif explainer == "lime":
            temp_explainer = MyLime(lime_models, choice, mode='regression')
            temp_vals = temp_explainer.attributions(choice)
        elif explainer == "numeric":
            temp_explainer = NumericExplainer(model, duffing.features, duffing.labels, h = 0.001)
            temp_vals = temp_explainer.feature_att(choice)
        else:
            print("not a valid explainer type")
        big_df = big_df.append(duffing.vals_to_df(temp_vals, choice, explainer = explainer, suffix = suffix))


    big_df.to_csv("Results/explainer_dataframe_"+suffix+".csv")  



Generating Data…: 100%|██████████████████| 100/100 [00:02<00:00, 35.57it/s]


kernel


  0%|          | 0/3 [00:00<?, ?it/s]

lime


Calculating Lime…: 100%|█████████████████████| 3/3 [00:47<00:00, 15.76s/it]


numeric


In [11]:
big_df

,xt_x0,xt_v0,xt_t,vt_x0,vt_v0,vt_t,x0,v0,t,explainer
0,0.046514,-0.000241,-0.445206,0.004180,-0.000358,-0.836986,0.075147,0.293592,0.469880,kernel
1,0.007293,0.020603,0.584114,-0.009046,0.009454,0.587466,0.698765,0.708605,0.638554,kernel
2,0.003159,0.001306,0.440204,-0.023578,0.000923,-0.507887,0.457968,0.204524,0.963855,kernel
0,-0.005003,-0.008927,-0.918356,-0.050988,-0.009415,-0.839964,0.075147,0.293592,0.469880,lime
1,-0.019613,0.006858,0.615167,-0.013609,-0.003278,1.240631,0.698765,0.708605,0.638554,lime
2,0.013448,-0.000509,0.366897,-0.049599,0.031390,-0.496008,0.457968,0.204524,0.963855,lime
0,-0.047989,-0.022839,-85.327895,-0.082123,-0.012337,8.154158,0.075147,0.293592,0.469880,numeric
1,-0.001347,-0.000938,32.417346,0.010879,-0.019797,-115.345908,0.698765,0.708605,0.638554,numeric
2,-0.048146,0.065927,-60.184014,0.019222,-0.003457,-75.627348,0.457968,0.204524,0.963855,numeric


In [12]:
temp_explainer = MyLime(lime_models, choice, mode='regression')

In [13]:
temp_explainer.kernel_width

AttributeError: 'MyLime' object has no attribute 'kernel_width'

In [14]:
np.sqrt(choice.shape[1]) * .75

1.299038105676658

In [15]:
choice.shape[1]

3